# SQLAlchemy-Mutable examples

## SQAlchemy setup

In [1]:
# from sqlalchemy_mutable import Mutable, MutableType, MutableModelBase, Query

# from sqlalchemy import Column, Integer, String, PickleType, create_engine 
# from sqlalchemy.orm import sessionmaker, scoped_session
# from sqlalchemy.ext.declarative import declarative_base

# # create a session (standard)
# engine = create_engine('sqlite:///:memory:')
# session_factory = sessionmaker(bind=engine)
# Session = scoped_session(session_factory)
# session = Session()
# Base = declarative_base()

# # subclass `MutableModelBase` when creating database models 
# # which may be stored in a `Mutable` object
# class MyModel(MutableModelBase, Base):
#     __tablename__ = 'mymodel'
#     id = Column(Integer, primary_key=True)
#     greeting = Column(String)
    
#     # initialize a database column with `MutableType`
#     mutable = Column(MutableType)
#     # add a `query` class attribute initialized with a scoped_session
#     # not necessary for use with Flask-SQLAlchemy
#     query = Query(Session) 
    
#     def __init__(self):
#         # set mutable column to `Mutable` object
#         self.mutable = Mutable()

# # create the database (standard)
# Base.metadata.create_all(engine)

## Flask-SQLAlchemy setup

In [2]:
from sqlalchemy_mutable import Mutable, MutableType, MutableModelBase

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# create a session (standard)
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# subclass `MutableModelBase` when creating database models
class MyModel(MutableModelBase, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    greeting = db.Column(db.String)
    
    # initialize a database column with `MutableType`
    mutable = db.Column(MutableType)  
    
    def __init__(self):
        # set mutable column to `Mutable` object
        self.mutable = Mutable()

# create the database (standard)
db.create_all()
session = db.session

## Index page examples

In [3]:
model = MyModel()
session.add(model)
session.commit()

# nested mutable objects
model.mutable.nested_mutable = Mutable()
session.commit()
model.mutable.nested_mutable.greet = 'hello world'
session.commit()
print(model.mutable.nested_mutable.greet)

# nested mutable list and dict
model.mutable = {}
session.commit()
model.mutable['greet'] = ['hello world']
session.commit()
print(model.mutable)

# storing database models
model.mutable = model
session.commit()
print(model.mutable)

# common literals
model.mutable = 'hello world'
session.commit()
print(model.mutable)

hello world
{'greet': ['hello world']}
<MyModel 1>
hello world


## Basic objects examples

In [4]:
model = MyModel()
session.add(model)
model.mutable.nested_mutable = Mutable()
session.commit()
# if `MyModel.mutable` weren't a `MutableType` column,
# this change would not survive a commit
model.mutable.nested_mutable.greeting = 'hello, world!'
session.commit()
model.mutable.nested_mutable.greeting

'hello, world!'

In [5]:
class MyClass():
    def greet(self, name='world'):
        return 'hello, {}!'.format(name)

@Mutable.register_coerced_type(MyClass)
class CoercedMyClass(Mutable, MyClass):
    pass

model = MyModel()
# without registering an associated coerced type,
# this will throw an error
model.mutable = MyClass()
model.mutable.greet()

'hello, world!'

In [6]:
class MyClass():
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return 'hello, {}!'.format(self.name)
    
@Mutable.register_tracked_type(MyClass)
class MutableMyClass(MyClass, Mutable):
    def __init__(self, source=None, root=None):
        '''
        Parameters
        ----------
        source : MyClass
            Original instance of `MyClass`. This will be converted into a `MutableMyClass` object.
            
        root : Mutable or None, default=None
            Root mutable object. This is handled by SQLAlchemy-Mutable. Set to `None` by default.
        '''
        super().__init__(name=source.name)
        
model = MyModel()
session.add(model)
model.mutable = Mutable()
model.mutable.object = MyClass('world')
session.commit()
model.mutable.object.name = 'moon'
session.commit()
model.mutable.object.greet()

'hello, moon!'

## Coerced types

In [7]:
import datetime

model = MyModel()
model.mutable = complex(1,1)
model.mutable = 1.
model.mutable = 1
model.mutable = 'hello world'
# model.mutable = datetime.datetime.now()
model.mutable

'hello world'

## Storing models (model shell)

In [8]:
model0 = MyModel()
model1 = MyModel()
session.add_all([model0, model1])
session.commit()
model0.mutable = model1
# without subclassing MutableModelBase,
# this would not retrieve `model1`
model0.mutable

<MyModel 5>

In [9]:
from sqlalchemy_mutable.model_shell import ModelShell

model = MyModel()
session.add(model)
session.commit()
shell = ModelShell(model)
shell == model

True

In [10]:
model = MyModel()
session.add(model)
session.commit()
model.mutable = {}
model.mutable['model'] = model
model.mutable.unshell()

{'model': <MyModel 7>}

## Mutable list and dict

In [11]:
model = MyModel()
model.mutable = []
session.add(model)
session.commit()
# without a mutable list,
# this change will not survive a commit
model.mutable.append('hello world')
session.commit()
model.mutable

['hello world']

In [12]:
model = MyModel()
model.mutable = {}
session.add(model)
session.commit()
# without a mutable dictionary,
# this change will not survive a commit
model.mutable['hello'] = 'world'
session.commit()
model.mutable

{'hello': 'world'}

## Type conversion

In [13]:
@Mutable.register_tracked_type(list) 
class MutableList(Mutable, list):
    def __init__(self, source=[], root=None):
        # 1. convert potentially mutable attributes/items to Mutable objects
        converted_list = self._convert_iterable(source)
        super().__init__(converted_list)
    
    # 2. classes with mutable items must have a `_tracked_items` attribute
    # `_tracked_items` is a list of potentially mutable items
    @property
    def _tracked_items(self):
        return list(self)
    
    # 3. call `self._changed()` to register change with the root Mutable object
    def append(self, item):
        self._changed()
        super().append(self._convert_item(item))
        
model = MyModel()
model.mutable = []
session.add(model)
session.commit()
# without using a mutable list, this change would not survive a commit
model.mutable.append('hello world')
session.commit()
model.mutable

['hello world']